# Simple Decoder, Version 6

From Karpathy's [Let's build GPT: from scratch, in code, spelled out.](https://www.youtube.com/watch?v=kCc8FmEb1nY&t=6096s), at time 1:26:34.

And now we complete the implementation for a Decoder model.

#### NOTES

1. adding in multiple block
1. residual connections
1. layer norm
1. scaling up the model, adding Dropout

In [1]:
import torch
import torch.nn as nn
from torch.nn import functional as F

In [2]:
# hyperparameters
batch_size = 64
block_size = 256
max_iters = 5000
eval_interval = 500
learning_rate = 3e-4
device = "cuda" if torch.cuda.is_available() else "cpu"
eval_iters = 200
n_embed = 384
n_head = 6
n_layer = 6
dropout = 0.2
# --------------


with open("input.txt", "r", encoding="UTF-8") as fin:
    text = fin.read()

# vocabulary of individual chars seen in text
chars = sorted(list(set(text)))
vocab_size = len(chars)

# character-level tokenization for the character-based transformer
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s]           # encoder: take a string, output a list of int
decode = lambda l: ''.join([itos[i] for i in l])  # decoder: take a list of int, output a string

# train and test splits, 90:10
data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9*len(data))
train_data = data[:n]
valid_data = data[n:]


def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == "train" else valid_data
    
    ix = torch.randint(len(data) - block_size, (batch_size,))
    
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    
    x, y = x.to(device), y.to(device)
    
    return x, y


@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ["train", "valid"]:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X,Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [3]:
%%time

# for reproducibility
torch.manual_seed(1337)


class Head(nn.Module):
    
    def __init__(self, head_size):
        super().__init__()
        self.query = nn.Linear(n_embed, head_size, bias=False)
        self.key = nn.Linear(n_embed, head_size, bias=False)
        self.value = nn.Linear(n_embed, head_size, bias=False)
        self.register_buffer("tril", torch.tril(torch.ones(block_size, block_size)))
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B,T,C = x.shape
        q = self.query(x)
        k = self.key(x)
        
        wei = q @ k.transpose(-2,-1) * C**-0.5
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float("-inf"))
        wei = F.softmax(wei, dim=-1)
        wei = self.dropout(wei)
        
        v = self.value(x)
        out = wei @ v
        return out

    
class MultiHeadAttention(nn.Module):
    """ Multiple heads of self-attention in parallel """
    
    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(n_embed, n_embed)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out
        

class FeedForward(nn.Module):
    """ If Self-Attention is the communication between tokens,
        the subsequent Feed-Forward layer allows the tokens to consider
        what they've learned through communication!
    """

    def __init__(self, n_embed):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embed, 4*n_embed),
            nn.ReLU(),
            nn.Linear(4*n_embed, n_embed),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)

    
class Block(nn.Module):
    """ Transformer block: communication followed by computation """

    def __init__(self, n_embed, n_head):
        super().__init__()
        head_size = n_embed // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedForward(n_embed)
        self.ln1 = nn.LayerNorm(n_embed)
        self.ln2 = nn.LayerNorm(n_embed)
        
    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x
    
        
class SimpleDecoderModel(nn.Module):
    
    def __init__(self):
        super().__init__()
        
        # each token directly reads off the logits
        # for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, n_embed)
        self.position_embedding_table = nn.Embedding(block_size, n_embed)
        self.blocks = nn.Sequential(*[Block(n_embed, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embed)
        self.lm_head = nn.Linear(n_embed, vocab_size)
 
    def forward(self, idx, targets=None):
        B, T = idx.shape
        
        # idx and targets are both (B,T) tensor of ints
        token_embeddings = self.token_embedding_table(idx)    # (B,T,C)
        position_embeddings = self.position_embedding_table(torch.arange(T, device=device)) #  (T,C)
        x = token_embeddings + position_embeddings            # (B,T,C)
        x = self.blocks(x)                                    # (B,T,C)
        x = self.ln_f(x)                                      # (B,T,C)
        logits = self.lm_head(x) # (B,T,vocab_size)
        
        if targets is None:
            loss = None
        else:
            B,T,C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        
        return logits, loss
    
    def generate(self, idx, max_new_tokens):
        # idx is (B,T) array of indices in the current context
        for _ in range(max_new_tokens):
            # 
            idx_cond = idx[:, -block_size:]
            
            # get predictions
            logits, loss = self(idx_cond)
            
            # focus only on the last time step (bigram)
            logits = logits[:, -1, :] # becomes (B, c)
            
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx 


model = SimpleDecoderModel()
m = model.to(device)

# create a PyTorch optimizer
optimizer = torch.optim.AdamW(m.parameters(), lr=learning_rate)

for iter in range(max_iters):
    
    # every once in a while, evaluate the loss on train and validation sets
    if iter % eval_interval == 0:
        losses = estimate_loss()
        print(f"step {iter}: train loss is {losses['train']:.4f}, validation loss {losses['valid']:.4f}")
    
    # sample a batch of data
    xb, yb = get_batch("train")
    
    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()


# generate from the model
context = torch.zeros((1,1), dtype=torch.long, device=device)
print(decode(m.generate(context, max_new_tokens=500)[0].tolist()))
print("==========================")
print()

step 0: train loss is 4.2849, validation loss 4.2824
step 500: train loss is 2.0035, validation loss 2.0879
step 1000: train loss is 1.5980, validation loss 1.7744
step 1500: train loss is 1.4408, validation loss 1.6337
step 2000: train loss is 1.3436, validation loss 1.5671
step 2500: train loss is 1.2828, validation loss 1.5327
step 3000: train loss is 1.2298, validation loss 1.5066
step 3500: train loss is 1.1866, validation loss 1.4868
step 4000: train loss is 1.1494, validation loss 1.4856
step 4500: train loss is 1.1098, validation loss 1.4778

But with prison: I will seek think you,
And some must sorry that you have bore.

ROMEO:
'Tis self more: if the hand, no more honour,
But by perfect more slaughter, and will the war.

ROMEO:
I have bore a clowled man: burief accepassade man.

BENVOLIO:
Is the heart of the sancture could prove it to flesh.
For Rome is do near not for thy rote?
Farewarring me out.

ROMEO:
Else, look thence:--
Wilt thou promise,--you call may not what prid
Thi